# Notebook 2: EDA y Preparación de Datos

## Objetivo
Análisis exploratorio de datos (EDA) y preparación de los datos para el cálculo de señales Momentum. Incluye cálculo de retornos logarítmicos mensuales y construcción del calendario de rebalanceo.

## Índice
1. [Configuración y Carga de Datos](#configuracion)
2. [Análisis Exploratorio de Datos](#eda)
3. [Cálculo de Retornos Logarítmicos](#retornos)
4. [Construcción del Calendario de Rebalanceo](#calendario)
5. [Preparación de Datos para Backtesting](#preparacion)

---

## 1. Configuración y Carga de Datos {#configuracion}

Importación de librerías y carga de datos procesados del Notebook 1.

In [ ]:
# Librerías permitidas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os
from datetime import datetime

# Configuración
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Cargar datos del Notebook 1
data_dir = '../data'

# Cargar datos procesados (descomentar cuando estén disponibles)
# price_data_clean = pd.read_parquet(f'{data_dir}/price_data_clean.parquet')
# prices_monthly = pd.read_parquet(f'{data_dir}/prices_monthly.parquet')
# universe_monthly = pd.read_csv(f'{data_dir}/universe_monthly.csv', index_col=0, parse_dates=True)

print("⚠️  Cargar datos del Notebook 1 antes de continuar")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Análisis Exploratorio de Datos {#eda}

Análisis de la estructura de datos, distribución de precios, valores faltantes y estadísticas descriptivas.

In [ ]:
def exploratory_data_analysis(df, title="Datos de Precios"):
    """
    Realiza análisis exploratorio básico de datos.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con datos a analizar
    title : str
        Título para los gráficos
    """
    print(f"\n=== ANÁLISIS EXPLORATORIO: {title} ===\n")
    
    # Información básica
    print("1. INFORMACIÓN GENERAL:")
    print(f"   - Forma del DataFrame: {df.shape}")
    print(f"   - Período: {df.index.min()} a {df.index.max()}")
    print(f"   - Total de días: {len(df)}")
    print(f"   - Total de activos: {len(df.columns)}")
    
    # Valores faltantes
    print("\n2. VALORES FALTANTES:")
    missing_pct = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
    print(f"   - Activos con >10% faltantes: {(missing_pct > 10).sum()}")
    print(f"   - Activos con >50% faltantes: {(missing_pct > 50).sum()}")
    print(f"   - Porcentaje total de faltantes: {df.isna().sum().sum() / (len(df) * len(df.columns)) * 100:.2f}%")
    
    # Estadísticas descriptivas
    print("\n3. ESTADÍSTICAS DESCRIPTIVAS:")
    print(df.describe().T[['mean', 'std', 'min', 'max']].head(10))
    
    # Visualización de valores faltantes
    if df.isna().sum().sum() > 0:
        plt.figure(figsize=(14, 6))
        missing_matrix = df.isna()
        plt.imshow(missing_matrix.T, aspect='auto', cmap='viridis', interpolation='nearest')
        plt.xlabel('Días')
        plt.ylabel('Activos')
        plt.title(f'Mapa de Valores Faltantes - {title}')
        plt.colorbar(label='Faltante')
        plt.tight_layout()
        plt.show()
    
    return missing_pct


# Ejecutar EDA (descomentar cuando price_data_clean esté disponible)
# missing_stats = exploratory_data_analysis(price_data_clean, title="Precios Diarios")

print("⚠️  Ejecutar EDA después de cargar datos")

## 3. Cálculo de Retornos Logarítmicos {#retornos}

**IMPORTANTE:** Según el PDF, se deben utilizar retornos logarítmicos para calcular las señales Momentum.

La fórmula de retorno logarítmico es: $r_t = \\ln(P_t / P_{t-1}) = \\ln(P_t) - \\ln(P_{t-1})$

In [ ]:
def calculate_log_returns(prices, method='daily'):
    """
    Calcula retornos logarítmicos.
    
    Parámetros:
    -----------
    prices : pd.DataFrame
        DataFrame con precios (índice: fechas, columnas: activos)
    method : str
        'daily' para retornos diarios, 'monthly' para mensuales
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con retornos logarítmicos
    """
    # Calcular retornos logarítmicos: ln(P_t / P_{t-1})
    log_returns = np.log(prices / prices.shift(1))
    
    # Eliminar primera fila (NaN)
    log_returns = log_returns.dropna()
    
    if method == 'monthly':
        # Agregar a mensual (suma de retornos logarítmicos diarios)
        log_returns_monthly = log_returns.resample('M').sum()
        return log_returns_monthly
    else:
        return log_returns


def calculate_monthly_returns_from_monthly_prices(monthly_prices):
    """
    Calcula retornos logarítmicos mensuales directamente desde precios mensuales.
    
    Parámetros:
    -----------
    monthly_prices : pd.DataFrame
        Precios mensuales (último día del mes)
    
    Retorna:
    --------
    pd.DataFrame
        Retornos logarítmicos mensuales
    """
    # Retorno logarítmico mensual: ln(P_mes_t / P_mes_{t-1})
    monthly_log_returns = np.log(monthly_prices / monthly_prices.shift(1))
    
    # Eliminar primera fila (NaN)
    monthly_log_returns = monthly_log_returns.dropna()
    
    return monthly_log_returns


# Calcular retornos logarítmicos diarios
# log_returns_daily = calculate_log_returns(price_data_clean, method='daily')

# Calcular retornos logarítmicos mensuales (método 1: desde precios diarios)
# log_returns_monthly = calculate_log_returns(price_data_clean, method='monthly')

# Calcular retornos logarítmicos mensuales (método 2: desde precios mensuales)
# log_returns_monthly = calculate_monthly_returns_from_monthly_prices(prices_monthly)

# Verificación: los retornos mensuales deben ser aproximadamente la suma de los diarios
# print("\\n=== VERIFICACIÓN DE RETORNOS MENSUALES ===")
# print(f"Período: {log_returns_monthly.index.min()} a {log_returns_monthly.index.max()}")
# print(f"Total de meses: {len(log_returns_monthly)}")
# print(f"\\nPrimeros retornos mensuales:")
# print(log_returns_monthly.head())

print("⚠️  Calcular retornos logarítmicos después de cargar datos")

In [ ]:
def build_rebalancing_calendar(df, start_date='2015-01-01', end_date=None):
    """
    Construye calendario de rebalanceo mensual.
    
    El rebalanceo se realiza el último día hábil del mes.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con datos diarios (para determinar días hábiles)
    start_date : str
        Fecha de inicio del backtest
    end_date : str, optional
        Fecha de fin (si None, usa la última fecha disponible)
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con fechas de rebalanceo
    """
    # Filtrar por período de backtest
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date) if end_date else df.index.max()
    
    # Obtener último día hábil de cada mes
    rebalance_dates = []
    
    # Generar rango de meses
    current = start.replace(day=1)  # Primer día del mes de inicio
    
    while current <= end:
        # Último día del mes
        last_day = (current + pd.offsets.MonthEnd(1))
        
        # Encontrar último día hábil <= last_day
        month_data = df.loc[df.index <= last_day]
        if len(month_data) > 0:
            last_business_day = month_data.index.max()
            if last_business_day >= start:  # Solo incluir si es >= fecha inicio
                rebalance_dates.append(last_business_day)
        
        # Avanzar al siguiente mes
        current = (current + pd.offsets.MonthEnd(1)) + pd.Timedelta(days=1)
    
    rebalance_calendar = pd.DataFrame({
        'rebalance_date': rebalance_dates,
        'year': [d.year for d in rebalance_dates],
        'month': [d.month for d in rebalance_dates]
    })
    
    rebalance_calendar.set_index('rebalance_date', inplace=True)
    
    return rebalance_calendar


# Construir calendario de rebalanceo
# rebalance_calendar = build_rebalancing_calendar(
#     price_data_clean,
#     start_date='2015-01-01',
#     end_date=None  # Hasta la actualidad
# )

# print("\\n=== CALENDARIO DE REBALANCEO ===")
# print(f"Total de fechas de rebalanceo: {len(rebalance_calendar)}")
# print(f"Primera fecha: {rebalance_calendar.index.min()}")
# print(f"Última fecha: {rebalance_calendar.index.max()}")
# print("\\nPrimeras 12 fechas:")
# print(rebalance_calendar.head(12))

print("⚠️  Construir calendario de rebalanceo después de cargar datos")

## 5. Preparación de Datos para Backtesting {#preparacion}

Preparación final de datos y guardado para uso en notebooks posteriores.

In [ ]:
# Guardar datos preparados para notebooks siguientes
# log_returns_monthly.to_parquet(f'{data_dir}/log_returns_monthly.parquet')
# rebalance_calendar.to_csv(f'{data_dir}/rebalance_calendar.csv')

# Visualización de distribución de retornos mensuales
# if 'log_returns_monthly' in locals():
#     fig, axes = plt.subplots(2, 2, figsize=(14, 10))
#     
#     # Seleccionar algunos activos representativos para visualización
#     sample_assets = log_returns_monthly.columns[:min(5, len(log_returns_monthly.columns))]
#     
#     # Distribución de retornos
#     for asset in sample_assets:
#         axes[0, 0].hist(log_returns_monthly[asset].dropna(), bins=50, alpha=0.5, label=asset)
#     axes[0, 0].set_xlabel('Retorno Logarítmico Mensual')
#     axes[0, 0].set_ylabel('Frecuencia')
#     axes[0, 0].set_title('Distribución de Retornos Mensuales')
#     axes[0, 0].legend()
#     axes[0, 0].grid(True, alpha=0.3)
#     
#     # Evolución temporal de retornos
#     for asset in sample_assets:
#         axes[0, 1].plot(log_returns_monthly.index, log_returns_monthly[asset], alpha=0.6, label=asset)
#     axes[0, 1].set_xlabel('Fecha')
#     axes[0, 1].set_ylabel('Retorno Logarítmico Mensual')
#     axes[0, 1].set_title('Evolución Temporal de Retornos')
#     axes[0, 1].legend()
#     axes[0, 1].grid(True, alpha=0.3)
#     
#     # Estadísticas por activo
#     stats_df = log_returns_monthly[sample_assets].describe().T
#     axes[1, 0].barh(stats_df.index, stats_df['std'])
#     axes[1, 0].set_xlabel('Volatilidad (Desviación Estándar)')
#     axes[1, 0].set_title('Volatilidad por Activo')
#     axes[1, 0].grid(True, alpha=0.3)
#     
#     # Matriz de correlación
#     corr_matrix = log_returns_monthly[sample_assets].corr()
#     sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, ax=axes[1, 1])
#     axes[1, 1].set_title('Matriz de Correlación')
#     
#     plt.tight_layout()
#     plt.show()

print("\\n=== RESUMEN DEL NOTEBOOK 2 ===")
print("✓ Análisis exploratorio de datos")
print("✓ Cálculo de retornos logarítmicos mensuales")
print("✓ Construcción del calendario de rebalanceo")
print("✓ Datos preparados para cálculo de señales")
print("\\n⚠️  IMPORTANTE: Ejecutar todas las celdas con datos reales")